# Przesukiwanie przestrzeni parametrów stukturalnych modelu za pomocą algorytmu optymalizacji dyskretnej


Dla każdego wejścia modelu `u_{i}` określamy wektor parametrów `(N_{i}, d_{i}, D_{i})`, gdzie:
- `N_{i}` - rząd dynamiki modelu, `N_{i} >= 0`,
- `d_{i}` - opóźnienie wejścia, `d_{i} >= 0`,
- `D_{i}` - rząd nieliniowości wejścia, `D_{i} > 0`

Strukturę modelu danego wyjścia `y_{j}` możemy opisać za pomocą macierzy M

`M = [(N_{1}, d_{1}, D_{1}), (N_{2}, d_{2}, D_{2}), ..., (N_{m}, d_{m}, D_{m})]`

Struktura macierzy M może zostać dobrana w drodze optymalizacji dyskretnej. Dla danej macierzy M' można zidentyfikować parametry dpowiadające modelowi, przy użyciu metody najmniejszych kwadratów.

## Zaimportuj potrzebne biblioteki

In [71]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.optimize import differential_evolution 

print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("matplotlib version: {}".format(mpl.__version__))

pandas version: 1.0.1
numpy version: 1.18.2
matplotlib version: 3.2.0


## Przygotuj dane uczące i weryfikacyjne


In [2]:
def changeDateToSeconds(df):
    first = df["date"][0]
    df["date"] = df["date"].apply(lambda timestamp: (timestamp-first).seconds)
    return df

def readDataFromExcel(path, sheet):
    df = pd.read_excel(path, sheet_name=sheet)
    df["date"] = pd.to_datetime(df["date"])
    df = changeDateToSeconds(df)
    return df

df_learn = readDataFromExcel("./data/K-1_MI.xlsx", "d2")
df_verif = readDataFromExcel("./data/K-1_MI.xlsx", "d6")

inputs = ["FWF", "PTWT"]
outputs = ["LT01"]

u_learn = df_learn[inputs].to_numpy()
y_learn = df_learn[outputs].to_numpy()

u_verif = df_verif[inputs].to_numpy()
y_verif = df_verif[outputs].to_numpy()

## Przygotuj macierz A do zadania najmniejszych kwadratów

### Macierz dla pojedynczego wejścia

In [3]:
def createModelMatrixForSingleInput(data, order, delay, exponent):
    if(order < 0 or delay < 0 or exponent <= 0):
        raise AssertionError("Invalid structure parameter")
        
    samples = data.shape[0]
    widthCoefficient = (order + 1)*exponent
    heightAbsoluteTerm = order + delay
    
    A = np.zeros([samples - heightAbsoluteTerm, widthCoefficient])
    
    for j in range(order+1):
        for k in range(exponent):
            colIndex = j*exponent + k
            A[:, colIndex] = np.power(data[j+delay : samples-heightAbsoluteTerm+j+delay], k+1)
    
    return A

### Macierz dla wszystkich wejść obiektu

In [4]:
def createModelMatrix(data, M):
    if(M.shape[0] != 3):
        raise AssertionError("Invalid parameter vector size")

    if(M.shape[1] != data.shape[1]):
        raise AssertionError("Mismatched size of data and M vector")
        
    inputs = M.shape[1]
    height = data.shape[0]
    
    maxDelay = 0; maxOrder = 0
    for index, parameters in enumerate(M.T):
        order, delay, exponent = parameters
        if(order > maxOrder):
            maxOrder = order
        if(delay > maxDelay):
            maxDelay = delay
            
    A = np.empty(shape=(height-maxOrder-maxDelay, 0)) 
    for index, parameters in enumerate(M.T):
        # stworz macierz dla danego wejscia
        inputData = data[:, index]
        order, delay, exponent = parameters
        Ap = createModelMatrixForSingleInput(data[:, index], order, delay, exponent)
        
        # obetnij macierz - delay od góry, a order od dołu macierzy
        delayDiff = maxDelay-delay
        orderDiff = maxOrder-order
        baseHeight = Ap.shape[0]
        validA = Ap[delayDiff: baseHeight-orderDiff]
        
        # dodaj do akumulatora
        A = np.concatenate((A, validA), axis=1)
        
    return A

## Przykładowy model dla obiektu o dwóch wejściach

In [5]:
# model structure
M = np.array([[5,0,1],[0,5,1]]).transpose()

# prepare outputs
maxDelay = 0; maxOrder = 0
for index, parameters in enumerate(M.T):
        order, delay, exponent = parameters
        if(order > maxOrder):
            maxOrder = order
        if(delay > maxDelay):
            maxDelay = delay
            
numberOfSamples = y_learn.shape[0]
output_learn_cut = y_learn[maxDelay : numberOfSamples - maxOrder]
output_verif_cut = y_verif[maxDelay : numberOfSamples - maxOrder]
             
A_learn = createModelMatrix(u_learn, M)
A_verif = createModelMatrix(u_verif, M)

model = LinearRegression().fit(A_learn, output_learn_cut)

output_model_learn = model.predict(A_learn)
output_model_verif = model.predict(A_verif)
        
learn_score = r2_score(output_learn_cut, output_model_learn)
verif_score = r2_score(output_verif_cut, output_model_verif)
      
print("learn score: {}, verif score: {}".format(learn_score, verif_score))

learn score: 0.27061438546916117, verif score: -13.527323652935552


## Optymalizacja dyskretna struktury modelu

### Funkcja celu :: Macierz struktury -> Wynik modelu

In [6]:
def evalModelScore(M, u_learn, y_learn, u_verif, y_verif):
    maxDelay = 0; maxOrder = 0
    for index, parameters in enumerate(M.T):
            order, delay, exponent = parameters
            if(order > maxOrder):
                maxOrder = order
            if(delay > maxDelay):
                maxDelay = delay

    numberOfSamples = y_learn.shape[0]
    output_learn_cut = y_learn[maxDelay : numberOfSamples - maxOrder]
    output_verif_cut = y_verif[maxDelay : numberOfSamples - maxOrder]

    A_learn = createModelMatrix(u_learn, M)
    A_verif = createModelMatrix(u_verif, M)

    model = LinearRegression().fit(A_learn, output_learn_cut)
    output_model_verif = model.predict(A_verif)
    
    verif_score = r2_score(output_verif_cut, output_model_verif)
    
    return verif_score

In [44]:
# Przykład użycia
M1 = np.array([[5,0,1],[0,5,1]]).transpose()
M2 = np.array([[5,2,2],[1,2,1]]).transpose()
M3 = np.array([[3,4,1],[2,1,2]]).transpose()

print(evalModelScore(M1, u_learn, y_learn, u_verif, y_verif))
print(evalModelScore(M2, u_learn, y_learn, u_verif, y_verif))
print(evalModelScore(M3, u_learn, y_learn, u_verif, y_verif))

-13.527323652935552
-99.77493416076736
-3.364357961219522


### Sformułuj zadanie optymalizacji

In [88]:
inputs = ["FWF","PTWT", "FW01", "FW02"]
outputs = ["LT01"]

u_learn = df_learn[inputs].to_numpy()
y_learn = df_learn[outputs].to_numpy()

u_verif = df_verif[inputs].to_numpy()
y_verif = df_verif[outputs].to_numpy()

def goalFunction(x):
    # zbuduj macierz M na podstawie wektora x
    M = np.empty(shape=(3, len(inputs)), dtype=np.int32)
    for index, value in enumerate(x):
        row = index % 3; col = index // 3
        M[row, col] = int(value)
      
    score = evalModelScore(M, u_learn, y_learn, u_verif, y_verif)
    return (-1)*score

### Znajdź parametry modelu

In [114]:
bounds = [(0, 10), (0,50), (1,5)]*len(inputs)

result = differential_evolution(goalFunction, bounds, disp=True, polish=False, maxiter=50, atol=1.0)

differential_evolution step 1: f(x)= 0.953283
differential_evolution step 2: f(x)= 0.953283
differential_evolution step 3: f(x)= 0.953283
differential_evolution step 4: f(x)= 0.953283
differential_evolution step 5: f(x)= 0.953283
differential_evolution step 6: f(x)= 0.623921
differential_evolution step 7: f(x)= 0.623921
differential_evolution step 8: f(x)= 0.623921
differential_evolution step 9: f(x)= 0.623921
differential_evolution step 10: f(x)= 0.623921
differential_evolution step 11: f(x)= 0.623921
differential_evolution step 12: f(x)= 0.623921
differential_evolution step 13: f(x)= 0.623921
differential_evolution step 14: f(x)= 0.623921
differential_evolution step 15: f(x)= 0.495213
differential_evolution step 16: f(x)= 0.495213
differential_evolution step 17: f(x)= 0.472734


In [113]:
optimM = np.empty(shape=(3, len(inputs)), dtype=np.int32)
for index, value in enumerate(result.x):
    row = index % 3; col = index // 3
    optimM[row, col] = int(value)
print(optimM)

[[ 0  1  5  3]
 [32 14 10 24]
 [ 2  1  3  1]]
